# Coroutines
A line such as yield item produces a value that is received by the caller of next(…), and it also gives way, suspending the execution of the generator so that the caller may proceed until it’s ready to consume another value by invoking next() again. A coroutine is syntactically like a generator: just a function with the yield keyword in its body. However, in a coroutine, yield usually appears on the right side of an expression (e.g., datum = yield), and it may or may not produce a value—if there is no
expression after the yield keyword, the generator yields None.  

In [7]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)


In [8]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x00000000051428E0>

In [9]:
import inspect
inspect.getgeneratorstate(my_coro)

'GEN_CREATED'

In [10]:
next(my_coro)

-> coroutine started


In [11]:
inspect.getgeneratorstate(my_coro)

'GEN_SUSPENDED'

It’s crucial to understand that the execution of the coroutine is suspended exactly at the yield keyword.

In [4]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [5]:
inspect.getgeneratorstate(my_coro)

'GEN_CLOSED'

## Example: Coroutine to Compute a Running Average

In [12]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count


In [14]:
coro_avg = averager()
coro_avg

<generator object averager at 0x0000000005142938>

In [15]:
inspect.getgeneratorstate(coro_avg)

'GEN_CREATED'

In [16]:
next(coro_avg)

In [17]:
inspect.getgeneratorstate(coro_avg)  # hits yield

'GEN_SUSPENDED'

In [18]:
coro_avg.send(10)  # yields average and waits to receive the next term

10.0

In [19]:
inspect.getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [20]:
coro_avg.send(20)

15.0

In [21]:
coro_avg.send(30)

20.0

## Decorators for Coroutine Priming
We can't do much with coroutines without priming them. We must always remember to call next(my_coro) before my_coro.send(x). 

In [22]:
from functools import wraps

def coroutine(func):
    ''' Decorator: primes 'func' by advancing to first 'yield' '''
    @wraps(func)
    def primer(*args,**kwargs):
        gen = func(*args,**kwargs)
        next(gen)
        return gen
    return primer


In [23]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count


In [25]:
avg = averager()
avg

<generator object averager at 0x0000000005142FC0>

In [27]:
avg.send(10)

10.0

In [28]:
avg.send(20)

15.0

In [29]:
avg.send(30)

20.0

## Coroutine Termination and Exception Handling

In [31]:
avg = averager()

In [32]:
avg.send(40)

40.0

In [33]:
avg.send('spam') 

TypeError: unsupported operand type(s) for +=: 'float' and 'str'